In [9]:
pip install ../

Processing c:\users\elba_ro\documents\projects\github\conf22-style-transfer
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for iesta: filename=iesta-1.3.59-py2.py3-none-any.whl size=22690 sha256=3983a005a75652e669ce4ea2a443911783ca00c010eb43b7a66cde330061f607
  Stored in directory: c:\users\elba_ro\appdata\local\pip\cache\wheels\77\fd\d3\cc84fb99fcf4d20c370ea3f838063a525bbcf56112a5076cd9
Successfully built iesta
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: iesta 1.3.59 has a non-standard dependency specifier nlpaf>==2.1.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of iesta or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:

import iesta.machine_learning.dataloader
#from .autonotebook import tqdm as notebook_tqdm
import iesta.stats.significance
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(3,2)}, 
    #style="white" # nicer layout
)


dotenv was False


In [2]:
feature_dfs= {}
_, feature_df = iesta.machine_learning.dataloader.load_training_features_df("Liberal")
feature_dfs["liberal"] = feature_df

_, c_feature_df = iesta.machine_learning.dataloader.load_training_features_df("Conservative")
feature_dfs["conservative"] = c_feature_df

iesta        INFO     File already created. Loading file...


2135: ../data/extracted_features//Liberal_style-features_5000\Liberal_batch5000_6_style-features.parquet
35: ../data/extracted_features//Liberal_transformer-features_100\Liberal_batch100_272_transformer-features.parquet


iesta        INFO     File already created. Loading file...


547: ../data/extracted_features//Conservative_style-features_5000\Conservative_batch5000_9_style-features.parquet
47: ../data/extracted_features//Conservative_transformer-features_100\Conservative_batch100_406_transformer-features.parquet


In [3]:
c_feature_df.columns.tolist()

['id',
 'debate_id',
 'p_name',
 'effect',
 'category',
 'round',
 'argument',
 'cleaned_text',
 'is_for_eval_classifier',
 'split',
 'mpqa_assessments',
 'mpqa_doubt',
 'mpqa_authority',
 'mpqa_emphasis',
 'mpqa_necessity',
 'mpqa_causation',
 'mpqa_generalization',
 'mpqa_structure',
 'mpqa_conditionals',
 'mpqa_inconsistency',
 'mpqa_possibility',
 'mpqa_wants',
 'mpqa_contrast',
 'mpqa_priority',
 'mpqa_difficulty',
 'mpqa_inyourshoes',
 'mpqa_rhetoricalquestion',
 'mpqa_argumentative',
 'mpqa_token_ratio',
 'mpqa_args_count',
 'empath_school_count',
 'empath_school_ratio',
 'empath_school_position',
 'empath_school_word',
 'empath_art_count',
 'empath_art_ratio',
 'empath_art_position',
 'empath_art_word',
 'empath_computer_count',
 'empath_computer_ratio',
 'empath_computer_position',
 'empath_computer_word',
 'empath_college_count',
 'empath_college_ratio',
 'empath_college_position',
 'empath_college_word',
 'empath_fear_count',
 'empath_fear_ratio',
 'empath_fear_position',
 '

In [5]:
significance = iesta.stats.significance.run_all_significance_test(feature_dfs)

Running {ideology} {iv} {excluded_effect}
Resampled dataset shape Counter({'effective': 1384, 'ineffective': 1384})
processed df length: 2768
data has 2768 instances
bonforrini_threshold:  0.05
there are 563 features.
Feature: mpqa_assessments
Feature: mpqa_doubt
Feature: mpqa_authority
Feature: mpqa_emphasis
Feature: mpqa_necessity
Feature: mpqa_causation
Feature: mpqa_generalization
Feature: mpqa_structure
Feature: mpqa_conditionals
Feature: mpqa_inconsistency
Feature: mpqa_possibility
Feature: mpqa_wants
Feature: mpqa_contrast
Feature: mpqa_priority
Feature: mpqa_difficulty
Feature: mpqa_inyourshoes
Feature: mpqa_rhetoricalquestion
Feature: mpqa_argumentative
Feature: mpqa_token_ratio
Feature: mpqa_args_count
Feature: empath_school_count
Feature: empath_school_ratio
Feature: empath_art_count
Feature: empath_art_ratio
Feature: empath_computer_count
Feature: empath_computer_ratio
Feature: empath_college_count
Feature: empath_college_ratio
Feature: empath_fear_count
Feature: empath_fea

In [60]:
for k in significance.keys():
    print(k)


def _get_key(
    ideology, disregard_okay=False, undersampled=False, binary_effect=False
):
    #_okay = "_Okay" if disregard_okay else ""
    _undersampled = "_undersampled" if undersampled else ""
    #_binary = "_binary" if binary_effect else ""
    _key = f"{ideology}_effect{_undersampled}_all_features"
    return _key


def _get_significance(
    ideology,
    #disregard_okay=False, 
    undersampled=False, 
    #binary_effect=False
    threshold: int = 0.03
):
    def _apply_absolute_effect(row):
        effect = row['ineffective effective'] 
        row['absolute_effect'] = effect*(-1) if effect < 0.0 else effect
        return row

    _key = _get_key(ideology, undersampled)
    _df : pd.DataFrame = significance[_key][0].reset_index().copy()
    _df = _df[
        (~_df["feature"].str.endswith("_count"))
       # & (~_df["feature"].str.endswith("_ratio"))
    ]
    if "Unnamed: 0" in _df.columns.tolist():
        del _df["Unnamed: 0"]
    _df = _df.apply(_apply_absolute_effect, axis=1)
    _df = _df.sort_values(by=['absolute_effect'], ascending=False)
    _df = _df[_df['absolute_effect']>=threshold]
    _df.set_index("feature", inplace=True)
    return _df

conservative_effect_undersampled_all_features
conservative_effect_all_features
liberal_effect_undersampled_all_features
liberal_effect_all_features


In [61]:
conservative_sign = _get_significance(
    "conservative"
)
liberal_sign = _get_significance(
    "liberal",)


,ineffective effective,absolute_effect
feature,,
liwc_OtherP,-0.06,0.06
liwc_AllPunc,-0.05,0.05
liwc_shehe,-0.05,0.05
liwc_WC,-0.05,0.05
liwc_polite,-0.04,0.04
liwc_male,-0.04,0.04
mpqa_argumentative,-0.04,0.04
liwc_prosocial,-0.04,0.04
liwc_i,-0.03,0.03


In [66]:
joint_sign = liberal_sign.join(conservative_sign, how='outer', lsuffix="_liberal", rsuffix="_cons")

In [67]:
joint_sign[:2]

,ineffective effective_liberal,absolute_effect_liberal,ineffective effective_cons,absolute_effect_cons
feature,,,,
emotion_hartmann_joy,-0.03,0.03,NaN,NaN
emotion_hartmann_joy_ratio,-0.03,0.03,-0.03,0.03


In [70]:
import pandas as pd
def _apply_ideology_effect(row):
    is_liberal =  not pd.isna(float(row['absolute_effect_liberal'])) 
    is_cons =  not pd.isna(float(row['absolute_effect_cons'])) 

    if is_liberal and is_cons:
        if row['ineffective effective_liberal'] *  row['ineffective effective_cons'] > 0.0:
            row['label'] = 'same'
        else: row['label'] = 'opposite'
    else: 
        row['label'] = 'conservative' if pd.isna(float(row['absolute_effect_liberal'])) else 'liberal'
    return row

joint_sign = joint_sign.apply(_apply_ideology_effect, axis = 1)

In [ ]:
joint_sign.sort_values(by=[""])